In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
#IMPORTING REQUIRED LIBRARIES

import pandas as pd
import numpy as np 
import os
import datetime

import matplotlib.pyplot as plt

import seaborn as sns 
sns.set_style('whitegrid')

import matplotlib

%matplotlib inline

### Loading data for Analysis

In [3]:
root_dir  = os.path.abspath('.')
data_dir  = os.path.join(root_dir, 'data')
train     = os.path.join(data_dir,'Train.csv')
test      = os.path.join(data_dir,'Test.csv' )
submt_fil = os.path.join(data_dir,'Final_submission.csv' )

In [4]:
train_df = pd.read_csv(train)
train_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284780 entries, 0 to 284779
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    284780 non-null  int64  
 1   StockCode    284780 non-null  int64  
 2   Description  284780 non-null  int64  
 3   Quantity     284780 non-null  int64  
 4   InvoiceDate  284780 non-null  object 
 5   UnitPrice    284780 non-null  float64
 6   CustomerID   284780 non-null  float64
 7   Country      284780 non-null  int64  
dtypes: float64(2), int64(5), object(1)
memory usage: 17.4+ MB


In [6]:
train_df.shape

(284780, 8)

In [7]:
train_df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [8]:
test_df = pd.read_csv(test)
test_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country
0,3139,1709,1079,3,2011-02-22 15:22:00,16161.0,35
1,617,510,3457,1,2010-12-08 12:46:00,17341.0,35
2,14653,604,694,36,2011-10-25 13:53:00,15158.0,35
3,8634,1478,3473,2,2011-06-27 12:38:00,16033.0,35
4,15546,3216,871,1,2011-11-06 16:14:00,15351.0,35


In [9]:
test_df.shape

(122049, 7)

In [10]:
test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122049 entries, 0 to 122048
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    122049 non-null  int64  
 1   StockCode    122049 non-null  int64  
 2   Description  122049 non-null  int64  
 3   Quantity     122049 non-null  int64  
 4   InvoiceDate  122049 non-null  object 
 5   CustomerID   122049 non-null  float64
 6   Country      122049 non-null  int64  
dtypes: float64(1), int64(5), object(1)
memory usage: 6.5+ MB


In [11]:
test_df.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
CustomerID     0
Country        0
dtype: int64

In [12]:
#Converting InvoiceDate column to datetime.

train_df['InvoiceDate'] = pd.to_datetime(train_df['InvoiceDate'], errors='coerce')
test_df['InvoiceDate'] = pd.to_datetime(test_df['InvoiceDate'], errors='coerce')

print(train_df.info())
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284780 entries, 0 to 284779
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    284780 non-null  int64         
 1   StockCode    284780 non-null  int64         
 2   Description  284780 non-null  int64         
 3   Quantity     284780 non-null  int64         
 4   InvoiceDate  284780 non-null  datetime64[ns]
 5   UnitPrice    284780 non-null  float64       
 6   CustomerID   284780 non-null  float64       
 7   Country      284780 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(5)
memory usage: 17.4 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122049 entries, 0 to 122048
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    122049 non-null  int64         
 1   StockCode    122049 non-null  int64         
 2   

In [13]:
monthMap = {1:'01',2:'02',3:'03',4:'04',5:'05',6:'06',7:'07',8:'08',9:'09',10:'10',11:'11',12:'12'}

# Extracting Month and Year from Date columns
train_df['Year'] = pd.DatetimeIndex(train_df['InvoiceDate']).year
train_df['Month'] = pd.DatetimeIndex(train_df['InvoiceDate']).month
#df.Month.apply(lambda x: datetime.strptime(str(x), '%m').strftime('%b'))
#train_df['MonthName'] = train_df.Month.apply(lambda x: datetime.datetime.strptime(str(x), '%m').strftime('%b'))
train_df['YearMonth'] = train_df["Year"].astype(str) + train_df["Month"].map(monthMap)
train_df['YearMonth'] = train_df['YearMonth'].apply(lambda x: int(x))

In [14]:
x = train_df.loc[train_df.UnitPrice >= 1000]
xGroupBy = x.groupby(["StockCode","Description","Country","YearMonth"]).agg({'UnitPrice':np.median}).reset_index()
xGroupBy.head()

,StockCode,Description,Country,YearMonth,UnitPrice
0,3678,882,35,201111,1038.75
1,3678,882,35,201112,1100.44
2,3679,1066,35,201104,1867.86
3,3680,1043,35,201111,1500.36
4,3680,1043,35,201112,1599.26


In [15]:
#updating Train_df outliers with median data 
#train_df = train_df.join(xGroupBy, on=["StockCode","Country"], how='left',rsuffix='_temp')
train_df = pd.merge(train_df, xGroupBy, on=["StockCode","Description","Country","YearMonth"], how='left')

In [16]:
train_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice_x,CustomerID,Country,Year,Month,YearMonth,UnitPrice_y
0,6141,1583,144,3,2011-05-06 16:54:00,3.75,14056.0,35,2011,5,201105,NaN
1,6349,1300,3682,6,2011-05-11 07:35:00,1.95,13098.0,35,2011,5,201105,NaN
2,16783,2178,1939,4,2011-11-20 13:20:00,5.95,15044.0,35,2011,11,201111,NaN
3,16971,2115,2983,1,2011-11-22 12:07:00,0.83,15525.0,35,2011,11,201111,NaN
4,6080,1210,2886,12,2011-05-06 09:00:00,1.65,13952.0,35,2011,5,201105,NaN


In [17]:
def myfunc(x,y):
    if x >= 1000:
        return y
    else:
        return x

In [18]:
train_df['UnitPrice'] = train_df.apply(lambda x: myfunc(x.UnitPrice_x, x.UnitPrice_y), axis=1)

In [19]:
monthMap = {1:'01',2:'02',3:'03',4:'04',5:'05',6:'06',7:'07',8:'08',9:'09',10:'10',11:'11',12:'12'}

# Extracting Month and Year from Date columns
test_df['Year'] = pd.DatetimeIndex(test_df['InvoiceDate']).year
test_df['Month'] = pd.DatetimeIndex(test_df['InvoiceDate']).month
test_df['YearMonth'] = test_df["Year"].astype(str) + test_df["Month"].map(monthMap)
test_df['YearMonth'] = test_df['YearMonth'].apply(lambda x: int(x))

In [20]:
train_df.drop(columns=["UnitPrice_x", "UnitPrice_y"], inplace=True)
train_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,CustomerID,Country,Year,Month,YearMonth,UnitPrice
0,6141,1583,144,3,2011-05-06 16:54:00,14056.0,35,2011,5,201105,3.75
1,6349,1300,3682,6,2011-05-11 07:35:00,13098.0,35,2011,5,201105,1.95
2,16783,2178,1939,4,2011-11-20 13:20:00,15044.0,35,2011,11,201111,5.95
3,16971,2115,2983,1,2011-11-22 12:07:00,15525.0,35,2011,11,201111,0.83
4,6080,1210,2886,12,2011-05-06 09:00:00,13952.0,35,2011,5,201105,1.65


### EDA and Feature Engineering on Training Dataset

In [21]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PowerTransformer

In [22]:
X = train_df.drop(columns=["InvoiceNo", 'InvoiceDate','CustomerID', 'Year','Month','UnitPrice'], axis =1)
y = train_df['UnitPrice']
test_df1 = test_df.copy()
test_df1 = test_df1.drop(columns=["InvoiceNo", 'InvoiceDate','CustomerID', 'Year','Month'], axis = 1)

labEncoder = LabelEncoder()
X['YearMonth'] = labEncoder.fit_transform(X['YearMonth'])
test_df1['YearMonth'] = labEncoder.transform(test_df1['YearMonth'])

powerTransformer = PowerTransformer()
qty = X['Quantity'].values.reshape(-1,1)
qty1 = test_df1["Quantity"].values.reshape(-1,1)
qty = powerTransformer.fit_transform(qty)
qty1 = powerTransformer.transform(qty1)
#Appending the transformed columns back to the data frame
X['Quantity'] = pd.DataFrame(data=qty)
test_df1['Quantity'] = pd.DataFrame(data=qty1)

minMaxStockCode = MinMaxScaler()
stockcode = X['StockCode'].values.reshape(-1,1)
stockcode1 = test_df1['StockCode'].values.reshape(-1,1)
stockcode = minMaxStockCode.fit_transform(stockcode)
stockcode1 = minMaxStockCode.transform(stockcode1)
#Appending the transformed columns back to the data frame
X['StockCode'] = pd.DataFrame(data=stockcode)
test_df1['StockCode'] = pd.DataFrame(data=stockcode1)

minMaxDesc = MinMaxScaler()
desc = X['Description'].values.reshape(-1,1)
desc1 = test_df1['Description'].values.reshape(-1,1)
desc = minMaxDesc.fit_transform(desc)
desc1 = minMaxDesc.transform(desc1)
#Appending the transformed columns back to the data frame
X['Description'] = pd.DataFrame(data=desc)
test_df1['Description'] = pd.DataFrame(data=desc1)

countryMinMax = MinMaxScaler()
cntry = X['Country'].values.reshape(-1,1)
cntry1 = test_df1['Country'].values.reshape(-1,1)
cntry = countryMinMax.fit_transform(cntry)
cntry1 = countryMinMax.transform(cntry1)
#Appending the transformed columns back to the data frame
X['Country'] = pd.DataFrame(data=cntry)
test_df1['Country'] = pd.DataFrame(data=cntry1)

output_power_trans = PowerTransformer()
y = y.values.reshape(-1,1)
y = output_power_trans.fit_transform(y)


In [23]:
X_train,X_test, y_train, y_test = train_test_split(X,y, train_size = .75, random_state = 42)   

In [24]:
from lightgbm import LGBMRegressor

In [25]:
lgbm = LGBMRegressor()

In [26]:
lgbm.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [29]:
lgbm.fit(X_train.values, y_train)
y_pred_train = lgbm.predict(X_train)
results = r2_score(y_train, y_pred_train)
print('train r2 score:', results)
results = mean_squared_error(y_train, y_pred_train)
print('train MSE:', results)

y_pred = lgbm.predict(X_test.values)
results = r2_score(y_test, y_pred)
print('Test r2 score:', results)
results = mean_squared_error(y_test, y_pred)
print('Test MSE:', results)

train r2 score: 0.7001145334325136
train MSE: 0.299961277371711
Test r2 score: 0.6959207068797835
Test MSE: 0.30382225085117626


In [30]:
finalPrediction = lgbm.predict(test_df1.values)

finalPrediction = output_power_trans.inverse_transform(finalPrediction.reshape(-1,1))
test_unitPrice = pd.DataFrame(data=finalPrediction,columns=["UnitPrice"])
test_unitPrice['UnitPrice'] = test_unitPrice['UnitPrice'].apply(lambda x: round(x,2))
test_unitPrice.head()

,UnitPrice
0,2.83
1,1.93
2,1.41
3,1.81
4,5.43


In [31]:
test_unitPrice.to_csv(os.path.join(data_dir,
                                   'Final_submission_XGBoost.csv' ), index=False)

In [36]:
params = {
    'num_leaves': [7, 14, 21, 28, 31, 50],
    'learning_rate': [0.1, 0.03, 0.003],
    'max_depth': [-1, 3, 5, 10,20],
    'n_estimators': [50, 100, 200, 500,1000],
    'min_child_samples':[10,20,30,50],
    'num_leaves':[2,8,16,32,64]
}

grid = GridSearchCV(LGBMRegressor(random_state=42), params, scoring='r2', cv=5)
grid.fit(X_train.values, y_train)

print(grid.best_score_)
print(grip.best_params_)

KeyboardInterrupt: 